<a href="https://colab.research.google.com/github/karencfisher/hotel-reviews/blob/main/notebooks/hotel_classification_quick_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!wget https://raw.githubusercontent.com/karencfisher/hotel-reviews/main/data/reviews_sample.csv

--2023-02-13 19:28:44--  https://raw.githubusercontent.com/karencfisher/hotel-reviews/main/data/reviews_sample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23866047 (23M) [text/plain]
Saving to: ‘reviews_sample.csv’

reviews_sample.csv  100%[===================>]  22.76M  --.-KB/s    in 0.1s    

2023-02-13 19:28:44 (155 MB/s) - ‘reviews_sample.csv’ saved [23866047/23866047]



In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import hamming_loss, accuracy_score 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
df_reviews = pd.read_csv('..\\data\\reviews_sample.csv')
df_reviews.drop(columns=['Overall', 'Location'], inplace=True)
df_reviews

,Cleanliness,Rooms,Service,Sleep Quality,Value,Content,average_score,sentiment
0,4,3,4,2,5,excellent value drawbacks second trip rius pla...,3.428571,0
1,2,3,2,3,3,great location hotel could better okay planned...,3.000000,0
2,3,2,2,2,3,slightly better dingy booked via hotelscom rea...,2.857143,0
3,5,2,2,5,2,could worse also much better bad hotel ever st...,3.142857,0
4,3,1,3,3,2,faded glory nice try place embarrassment marri...,2.571429,0
...,...,...,...,...,...,...,...,...
35117,5,3,1,4,5,say liked price cleaning service didnt pay muc...,3.571429,1
35118,4,4,4,4,5,affordable cleanm convenient touch atmosphere ...,4.142857,1
35119,5,5,5,5,5,great stay would gladly go back free copies ny...,5.000000,1
35120,5,5,5,5,5,resort hotel great value terrific place conven...,5.000000,1


Convert ratings to tags for Cleanliness, Rooms, Service, and Value. 1 if less than rating of 4 to indicate in need of attention.

In [4]:
tags = ['Cleanliness', 'Rooms', 'Service', 'Sleep Quality', 'Value']
df_reviews[tags].astype(int)

for tag in tags:
  df_reviews[tag] = df_reviews[tag].apply(lambda x: 1 if x < 4 else 0)

df_reviews

,Cleanliness,Rooms,Service,Sleep Quality,Value,Content,average_score,sentiment
0,0,1,0,1,0,excellent value drawbacks second trip rius pla...,3.428571,0
1,1,1,1,1,1,great location hotel could better okay planned...,3.000000,0
2,1,1,1,1,1,slightly better dingy booked via hotelscom rea...,2.857143,0
3,0,1,1,0,1,could worse also much better bad hotel ever st...,3.142857,0
4,1,1,1,1,1,faded glory nice try place embarrassment marri...,2.571429,0
...,...,...,...,...,...,...,...,...
35117,0,1,1,0,0,say liked price cleaning service didnt pay muc...,3.571429,1
35118,0,0,0,0,0,affordable cleanm convenient touch atmosphere ...,4.142857,1
35119,0,0,0,0,0,great stay would gladly go back free copies ny...,5.000000,1
35120,0,0,0,0,0,resort hotel great value terrific place conven...,5.000000,1


Test with a small sample of the data set

In [5]:
y = np.asarray(df_reviews[tags])
x_train, x_test, y_train, y_test = train_test_split(df_reviews['Content'], 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
x_train.shape, y_train.shape

((24585,), (24585, 5))

Construct a consistent pipeline. 

In [6]:
class Model:
  def __init__(self, input_shape):
    self.model = Sequential([Dense(5, 
                             activation='sigmoid', 
                             input_shape=input_shape)])

  def fit(self, x, y, epochs=100):
    self.model.compile(optimizer='adam', loss='mse')
    history=self.model.fit(x, y, epochs=epochs)
    return history

  def predict(self, x):
    return self.model.predict(x)


class CLF:
  def __init__(self, max_features):
    self.vectorize = TfidfVectorizer(max_features=max_features)
    self.clf = Model((max_features,))

  def fit(self, x, y, epochs=100):
    self.vectorize.fit(x)
    x = self.vectorize.transform(x).toarray()
    self.clf.fit(x, y, epochs)
    return self, x

  def predict(self, x, threshold=0.5):
    x = self.vectorize.transform(x).toarray()
    logits =  self.clf.predict(x)
    y = (logits > threshold).astype(int)
    return y, logits


In [7]:
pipe = CLF(5)
_, x = pipe.fit(x_train, y_train, epochs=20)

Epoch 1/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2413
Epoch 2/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2272
Epoch 3/20
769/769 [==============================] - 1s 995us/step - loss: 0.2225
Epoch 4/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2205
Epoch 5/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2196
Epoch 6/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2193
Epoch 7/20
769/769 [==============================] - 1s 979us/step - loss: 0.2191
Epoch 8/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2190
Epoch 9/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2190
Epoch 10/20
769/769 [==============================] - 1s 976us/step - loss: 0.2190
Epoch 11/20
769/769 [==============================] - 1s 1ms/step - loss: 0.2189
Epoch 12/20
769/769 [==============================] - 1s 983us/step - loss: 0.2189
Epoch 13/20
769/7

In [8]:
y_hat, logits = pipe.predict(x_test)
print(logits, y_hat)

330/330 [==============================] - 0s 918us/step
[[0.59182394 0.75796014 0.70229465 0.7038908  0.7601458 ]
 [0.582668   0.74973935 0.69631606 0.6966466  0.75363874]
 [0.29927838 0.46171412 0.39443293 0.41572273 0.4350213 ]
 ...
 [0.57118523 0.76812106 0.6777133  0.6950463  0.73411375]
 [0.37988624 0.5227125  0.4915803  0.4746301  0.54098266]
 [0.26837507 0.40504482 0.35964358 0.36759296 0.4140962 ]] [[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 ...
 [1 1 1 1 1]
 [0 1 0 0 1]
 [0 0 0 0 0]]


In [9]:
y_test[:5]

array([[0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0]], dtype=int64)

In [10]:
x_test[:5]

4520     one worst rooms ive stayed booked hotel basis ...
13214    mixed feelings im frequent traveler business l...
5706     would great stay except wrong charges bad serv...
29850    good option staff friendly otherwise good leve...
34791    good value clean comfortable hotel located rig...
Name: Content, dtype: object

In [11]:
print(accuracy_score(y_test, y_hat)) 
print(hamming_loss(y_test, y_hat))

0.3128974091297333
0.35380089209452403


In [12]:
myreview = "This is the niceset hotel I have stayed in. The staff was really helpful and kind. The room was spotless. I will recommend this to all my friends!"
result = pipe.predict([myreview])
result

1/1 [==============================] - 0s 19ms/step


(array([[1, 1, 1, 1, 1]]),
 array([[0.5464768 , 0.7154078 , 0.6687941 , 0.6648075 , 0.72473246]],
       dtype=float32))

In [64]:
badreview = "I had no sleep at all. The room was filthy. The service really did not care at all."
result = pipe.predict([badreview])
result

(array([[1, 1, 1, 1, 1]]),
 array([[0.6717682 , 0.7540729 , 0.7506741 , 0.9095476 , 0.91178936]],
       dtype=float32))

In [65]:
tags

['Cleanliness', 'Rooms', 'Service', 'Sleep Quality', 'Value']

In [66]:
review = "A very nice place except the service is poor."
result = pipe.predict([badreview])
result

(array([[1, 1, 1, 1, 1]]),
 array([[0.6717682 , 0.7540729 , 0.7506741 , 0.9095476 , 0.91178936]],
       dtype=float32))